In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
!pip install s3fs

  Using cached s3fs-2022.11.0-py3-none-any.whl (27 kB)
  Using cached aiohttp-3.8.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached aiobotocore-2.4.2-py3-none-any.whl (66 kB)
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 kB)
  Using cached botocore-1.27.59-py3-none-any.whl (9.1 MB)
  Using cached aioitertools-0.11.0-py3-none-any.whl (23 kB)
  Using cached wrapt-1.14.1-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (77 kB)
  Using cached frozenlist-1.3.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (158 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached multidict-6.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.8.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (264 kB)
  Using cached jmespath-1.0.1-py3-none-a

In [ ]:
plans = pd.read_csv('2018/plans.csv.gz')

In [6]:
personlist = list(plans[plans['trip_mode'] == 'WALK']['person_id'].unique())

In [8]:
filePath = "s3://beam-outputs/output/sfbay/sfbay-pilates-base__2022-12-20_09-38-17_jqa/"
data = pd.read_csv(filePath + 'ITERS/it.0/0.events.csv.gz')

/tmp/ipykernel_250/3367400546.py:2: DtypeWarning: Columns (0,1,5,18,19,21,22,23,24,28,29,32,34,35,39,41,43,45,46,47,48,50,51,52,53,54,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filePath + 'ITERS/it.0/0.events.csv.gz')


In [14]:


# Define distance bins and bin names
binnames = {1: "0-1", 2: "1-2", 3: "2-5", 4: "5-15", 5: "15+"}
distanceBinsMiles = np.array([0, 1, 2, 5, 15, 500])

# Load data into events
events = data

# Define function to fix PathTraversal data
def fixPathTraversals(PTs):
    PTs['duration'] = PTs['arrivalTime'] - PTs['departureTime']
    PTs['mode_extended'] = PTs['mode']
    PTs['isRH'] = PTs['vehicle'].str.contains('rideHail')
    PTs['isCAV'] = PTs['vehicleType'].str.contains('L5')
    PTs.loc[PTs['isRH'], 'mode_extended'] += '_RideHail'
    PTs.loc[PTs['isCAV'], 'mode_extended'] += '_CAV'
    PTs['occupancy'] = PTs['numPassengers']
    PTs.loc[PTs['mode_extended'] == 'car', 'occupancy'] += 1
    PTs.loc[PTs['mode_extended'] == 'walk', 'occupancy'] = 1
    PTs.loc[PTs['mode_extended'] == 'bike', 'occupancy'] = 1
    PTs['vehicleMiles'] = PTs['length'] / 1609.34
    PTs['passengerMiles'] = (PTs['length'] * PTs['occupancy']) / 1609.34
    PTs['totalEnergyInJoules'] = PTs['primaryFuel'] + PTs['secondaryFuel']
    PTs['gallonsGasoline'] = 0
    PTs.loc[PTs['primaryFuelType'] == 'gasoline',
            'gallonsGasoline'] += PTs.loc[PTs['primaryFuelType'] == 'gasoline', 'primaryFuel'] * 8.3141841e-9
    PTs.loc[PTs['secondaryFuelType'] == 'gasoline',
            'gallonsGasoline'] += PTs.loc[PTs['secondaryFuelType'] == 'gasoline', 'secondaryFuel'] * 8.3141841e-9
    PTs.drop(columns=['numPassengers', 'length'], inplace=True)
    return PTs

# Define function to update transit mode data
def updateTransitMode(MC):
    transit = MC['mode'] == 'walk_transit'
    railTrips = transit & MC['legModes'].str.contains("RAIL")
    ferryTrips = transit & ~railTrips & MC['legModes'].str.contains("FERRY")
    subwayTrips = transit & ~railTrips & ~ferryTrips & MC['legModes'].str.contains("SUBWAY")
    busTrips = transit & ~railTrips & ~ferryTrips & ~subwayTrips
    MC.loc[subwayTrips, 'mode'] = "subway"
    MC.loc[railTrips, 'mode'] = "rail"
    MC.loc[ferryTrips, 'mode'] = "ferry"
    MC.loc[busTrips, 'mode'] = "bus"
    MC['miles'] = MC['length'] / 1609.34
    MC['distBin'] = [binnames.get(val, 'Other') for val in np.digitize(MC['miles'], distanceBinsMiles)]
    return MC

PT = events.loc[(events['type'] == 'PathTraversal') & (events['length'] > 0)].dropna(how='all', axis=1)
MC = events.loc[(events['type'] == 'ModeChoice') & (events['length'] > 0) & (events['person'].isin(personlist))].dropna(how='all', axis=1)
AS = events.loc[(events['type'] == 'actstart')].dropna(how='all', axis=1)

del events

MC = updateTransitMode(MC)

PT['departureTime'] = PT['departureTime'].astype(int)
PT['arrivalTime'] = PT['arrivalTime'].astype(int)
PT = fixPathTraversals(PT)

gb = MC[['mode','person', 'distBin']].groupby(['mode', 'distBin']).agg('size').unstack().fillna(0.0)
gb.sum(axis=1).to_csv('out/total-trips-sfbay-may-4.csv')
tot = gb.sum(axis=0)
gb = gb / gb.sum(axis=0)
gb = gb / gb.sum(axis=0)
gb.loc['total',:] = tot/tot.sum()
gb.to_csv('out/mode-shares-sfbay-may-4.csv')
tripsPerPerson = AS[['person','actType']].groupby('actType').agg('size') / 300424
tripsPerPerson.to_csv('out/trips-percapita-sfbay-may-4.csv')
print("done")

done


In [15]:
len(MC['person'].unique())

496586

In [16]:
gb

distBin,0-1,1-2,15+,2-5,5-15
mode,,,,,
bike,0.055657,0.062874,0.000271,0.032755,0.004861
bike_transit,0.000002,0.000052,0.000648,0.000323,0.000808
bus,0.092225,0.099463,0.014198,0.077550,0.044381
car,0.254887,0.422869,0.874590,0.705645,0.838804
drive_transit,0.000000,0.000000,0.000022,0.000457,0.000670
ferry,0.000000,0.000000,0.000061,0.000000,0.000000
rail,0.000000,0.000007,0.011683,0.000190,0.002592
ride_hail,0.000010,0.000019,0.002858,0.001294,0.002719
ride_hail_pooled,0.000043,0.000085,0.043729,0.000322,0.010414


In [17]:
gb = MC[['mode','person', 'distBin']].groupby(['mode', 'distBin']).agg('size').unstack().fillna(0.0)

In [18]:
lenPerson = len(MC['person'].unique())

In [19]:
gb

distBin,0-1,1-2,15+,2-5,5-15
mode,,,,,
bike,26895.0,33980.0,49.0,22206.0,2190.0
bike_transit,1.0,28.0,117.0,219.0,364.0
bus,44566.0,53754.0,2563.0,52575.0,19996.0
car,123169.0,228537.0,157881.0,478393.0,377929.0
drive_transit,0.0,0.0,4.0,310.0,302.0
ferry,0.0,0.0,11.0,0.0,0.0
rail,0.0,4.0,2109.0,129.0,1168.0
ride_hail,5.0,10.0,516.0,877.0,1225.0
ride_hail_pooled,21.0,46.0,7894.0,218.0,4692.0


In [20]:
table = gb/lenPerson

In [21]:
table.to_csv('out/percapita_mode2.csv')